In [ ]:
!pip install sentence-transformers

In [ ]:
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('drive/MyDrive/Fraud_cleaned.csv')

<ipython-input-6-86b5ec75d693>:1: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('drive/MyDrive/Fraud_cleaned.csv')


In [ ]:
df.head(5)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Forum,Forum_Name,Thread_Name,Post_content,Post_date,Forum_thread_count,Forum_message_count,Thread_reply_count,Thread_view_count,Post_user,Post_user_url,Forum_message_count.1
0,0,0,0,24rcbiz,Cryptocurrencies,Creating a Bitcoin wallet + Rating of cryptocu...,Здесь мне хотелось рассказать вам как создать ...,28.06.2019,86.0,1810.0,120.0,Просмотры\n12K,Rappoport,https://24rc.biz/members/rappoport.6159/,NaN
1,1,1,1,24rcbiz,Cryptocurrencies,Creating a Bitcoin wallet + Rating of cryptocu...,Rappoport написал(а):\n\n\nЗдесь мне хотелось ...,28.06.2019,86.0,1810.0,120.0,Просмотры\n12K,barin2010,https://24rc.biz/members/barin2010.14820/,NaN
2,2,2,2,24rcbiz,Cryptocurrencies,Creating a Bitcoin wallet + Rating of cryptocu...,"Yes, in my opinion it’s better to create a pay...",28.06.2019,86.0,1810.0,120.0,Просмотры\n12K,semi,https://24rc.biz/members/semil.8761/,NaN
3,3,3,3,24rcbiz,Cryptocurrencies,Creating a Bitcoin wallet + Rating of cryptocu...,Хранение криптовалюты — один из самых актуальн...,28.06.2019,86.0,1810.0,120.0,Просмотры\n12K,Rappoport,https://24rc.biz/members/rappoport.6159/,NaN
4,4,4,4,24rcbiz,Cryptocurrencies,Creating a Bitcoin wallet + Rating of cryptocu...,"everything is simple and clear, as for me",28.06.2019,86.0,1810.0,120.0,Просмотры\n12K,Serhii8102,https://24rc.biz/members/serhii8102.15650/,NaN


In [ ]:
def g(df):
    return df.loc[df['Post_content'].str.len() >= 100] # lenght metric including spaces
clean_df=g(df.copy())
clean_df.drop_duplicates(inplace=True)

In [ ]:
clean_df.reset_index(inplace=True)

In [ ]:
clean_df.shape

(818761, 16)

In [ ]:
clean_df['Forum'].nunique()

49

In [ ]:
# clean_df.to_csv("clean_df.csv")

In [ ]:
!pip install sentence_transformers

In [ ]:
clean_df=clean_df[['Post_content','Post_date','Forum','Post_user']]

In [ ]:
clean_df.to_csv("fraud_cleaned.csv")

In [ ]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('bert-base-nli-mean-tokens')
# embeddings = model.encode(clean_df['Post_content'].astype(str))

In [ ]:
#import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed(clean_df['Post_content'].astype(str))


In [ ]:
import numpy as np
np.save('embeddings.npy', embeddings)
loaded_embeddings = np.load('embeddings.npy')


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(embeddings)
threshold = 0.95
similar_texts = []

In [ ]:
similarity_matrix

In [ ]:
post1=[]
date1=[]
user1=[]
post2=[]
date2=[]
user2=[]

In [ ]:
similar_dates=[]
for i in range(len(clean_df)):
    for j in range(i+1, len(clean_df)):
        if similarity_matrix[i, j] >=0.95 and clean_df.iloc[i]['Post_time']!= clean_df.iloc[j]['Post_time'] and clean_df.iloc[i]['Post_user']!= clean_df.iloc[j]['Post_user'] and clean_df.iloc[i]['Forum']!= clean_df.iloc[j]['Forum'] :
            #similar_texts.append((clean_df.iloc[i]['Post_content'], clean_df.iloc[j]['Post_content']))
            similar_dates.append((clean_df.iloc[i]['Post_time'], clean_df.iloc[j]['Post_time']))
            #print("XXXXXXXXXXXXXXXXX     SIMILAR POSTINGS-1   XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            post1.append((clean_df.iloc[i]['Post_content']))
            #print("XXXXXXXXXXXXXXXXX     SIMILAR POSTINGS-2   XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            post2.append(clean_df.iloc[j]['Post_content'])
            #print("XXXXXXXXXXXXXXXXX     SIMILAR DATES OF POSTS   XXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            date1.append(clean_df.iloc[i]['Post_time'])
            date2.append(clean_df.iloc[j]['Post_time'])
            user1.append(clean_df.iloc[i]['Post_user'])
            user2.append(clean_df.iloc[j]['Post_user'])
            print(i,i)
            #print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    NEW ONE-{i}    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

In [ ]:
similar_texts=pd.DataFrame({'Post-1': post1, 'Date-1': date1,'User-1': user1, 'Post-2': post2, 'Date-2': date2, 'User-2': user2})
similar_texts.head(5)
similar_texts.to_csv("similar_texts_xss.csv")

In [ ]:
similar_texts.drop_duplicates(inplace=True)

,Post-1,Date-1,User-1,Post-2,Date-2,User-2
0,"You won’t sign up without an invite, because i...",24.09.2022,arsarsov,arsarsov said:\nYou won’t sign up without an i...,01.10.2022,Dwight149
4,"Look at the versions of apache, mysql, php. If...",01.10.2022,l1zzka,"l1zzka said:\nLook at the versions of apache, ...",02.10.2022,Riuji
8,May be this can help\n.....\ngive more details...,24.03.2022,captain1337,captain1337 ??????(?):\nMay be this can help\n...,31.03.2022,FROM
12,Thank you Shrek for such a detailed instructio...,14.08.2005,Vladik,Thank you Shrek for such a detailed instructio...,21.08.2005,kievman
16,"Hello fellow members , im looking for a experi...",17.12.2022,aziefaison,"aziefaison ??????(?):\nHello fellow members , ...",18.12.2022,kozak deex
...,...,...,...,...,...,...
400,"The topic is this, there is a backup of the Or...",17.12.2022,peterpeter,"petrpetrov said:\nThe topic is this, there is ...",18.12.2022,VIPteslo
404,bigheadguy said:\nyou think naively)) the crus...,10.11.2022,Gotcha0z,"gotcha0z said:\ntherefore, we are talking abou...",11.11.2022,ataman888
408,Gather me the entire collection of mail:pass d...,16.01.2021,SingleAdwice,SingleAdvice said:\nGather me the entire colle...,31.01.2021,magolom
412,"I don’t understand everything, with Gifts, if ...",08.12.2018,jess_hide,jess_hide said:\nI don’t understand everything...,21.01.2020,gerix19961


In [ ]:
similar_texts.shape

(420, 6)

In [ ]:
similar_texts.head(5)

,Post-1,Date-1,User-1,Post-2,Date-2,User-2
0,"You won’t sign up without an invite, because i...",24.09.2022,arsarsov,arsarsov said:\nYou won’t sign up without an i...,01.10.2022,Dwight149
1,"You won’t sign up without an invite, because i...",24.09.2022,arsarsov,arsarsov said:\nYou won’t sign up without an i...,01.10.2022,Dwight149
2,"You won’t sign up without an invite, because i...",24.09.2022,arsarsov,arsarsov said:\nYou won’t sign up without an i...,01.10.2022,Dwight149
3,"You won’t sign up without an invite, because i...",24.09.2022,arsarsov,arsarsov said:\nYou won’t sign up without an i...,01.10.2022,Dwight149
4,"Look at the versions of apache, mysql, php. If...",01.10.2022,l1zzka,"l1zzka said:\nLook at the versions of apache, ...",02.10.2022,Riuji


In [ ]:
similar_texts.to_csv("drive/MyDrive/Fraud_similar_posts.csv")